# **1-Loading DataSets**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
downtime_factors = pd.read_excel('Downtime factors.xlsx')
line_downtime = pd.read_excel('Line downtime.xlsx')
line_productivity = pd.read_excel('Line productivity.xlsx')
products = pd.read_excel('Products.xlsx')

# **2-Exploratory data analysis**

In [ ]:
downtime_factors.head()
line_downtime.head()
line_productivity.tail()
products.head()

,Product,Flavor,Size,Min batch time
0,OR-600,Orange,600 ml,60
1,LE-600,Lemon lime,600 ml,60
2,CO-600,Cola,600 ml,60
3,DC-600,Diet Cola,600 ml,60
4,RB-600,Root Berry,600 ml,60


In [ ]:
downtime_factors.dtypes
line_downtime.dtypes
line_productivity.dtypes
products.dtypes

,0
Product,object
Flavor,object
Size,object
Min batch time,int64


In [ ]:
downtime_factors.info()
line_downtime.info()
line_productivity.info()
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Factor          12 non-null     int64 
 1   Description     12 non-null     object
 2   Operator Error  12 non-null     object
dtypes: int64(1), object(2)
memory usage: 420.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Batch   38 non-null     int64  
 1   1       0 non-null      float64
 2   2       5 non-null      float64
 3   3       2 non-null      float64
 4   4       9 non-null      float64
 5   5       3 non-null      float64
 6   6       12 non-null     float64
 7   7       11 non-null     float64
 8   8       6 non-null      float64
 9   9       1 non-null      float64
 10  10      3 non-null      float64
 11  11      3 non-null      float64
 1

In [ ]:
downtime_factors.isnull().sum()
line_downtime.isnull().sum()
line_productivity.isnull().sum()
products.isnull().sum()

,0
Product,0
Flavor,0
Size,0
Min batch time,0


In [ ]:
print(line_productivity['Start Time'])
print(line_productivity['End Time'])

0     11:50:00
1     14:05:00
2     15:45:00
3     17:35:00
4     19:15:00
5     20:39:00
6     21:39:00
7     04:05:00
8     06:05:00
9     07:30:00
10    09:22:00
11    10:37:00
12    12:02:00
13    14:15:00
14    15:55:00
15    17:15:00
16    18:59:00
17    20:22:00
18    22:14:00
19    07:45:00
20    09:05:00
21    10:35:00
22    11:35:00
23    12:55:00
24    14:45:00
25    16:30:00
26    01:00:00
27    02:45:00
28    04:05:00
29    05:40:00
30    07:43:00
31    08:50:00
32    10:20:00
33    12:18:00
34    14:50:00
35    16:50:00
36    19:30:00
37    22:55:00
Name: Start Time, dtype: object
0     14:05:00
1     15:45:00
2     17:35:00
3     19:15:00
4     20:39:00
5     21:39:00
6     22:54:00
7     06:05:00
8     07:30:00
9     09:22:00
10    10:37:00
11    12:02:00
12    14:15:00
13    15:55:00
14    17:15:00
15    18:59:00
16    20:22:00
17    22:14:00
18    23:29:00
19    09:05:00
20    10:35:00
21    11:35:00
22    12:55:00
23    14:45:00
24    16:30:00
25    17:30:00
26    02

In [ ]:
print(line_productivity.duplicated().sum())

0


In [ ]:
missing_batches = set(line_downtime['Batch']) - set(line_productivity['Batch'])
print(f"Missing batches in productivity data: {missing_batches}")

Missing batches in productivity data: set()


In [ ]:
missing_products = set(line_productivity['Product']) - set(products['Product'])
print(f"Missing products in product data: {missing_products}")

Missing products in product data: set()


# **3-Data Cleaning and Preprocessing**

In [ ]:
line_downtime.fillna(0, inplace=True)

In [ ]:
line_productivity['Start Time'] = pd.to_datetime(line_productivity['Start Time'], format='%H:%M:%S')
line_productivity['End Time'] = pd.to_datetime(line_productivity['End Time'], format='%H:%M:%S')

In [ ]:
line_productivity['Start Time'] = pd.to_datetime(line_productivity['Date'].astype(str) + ' ' + line_productivity['Start Time'].astype(str))
line_productivity['End Time'] = pd.to_datetime(line_productivity['Date'].astype(str) + ' ' + line_productivity['End Time'].astype(str))
#Handle datetime Issues


<ipython-input-19-5cc7c2b44874>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  line_productivity['Start Time'] = pd.to_datetime(line_productivity['Date'].astype(str) + ' ' + line_productivity['Start Time'].astype(str))
<ipython-input-19-5cc7c2b44874>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  line_productivity['End Time'] = pd.to_datetime(line_productivity['Date'].astype(str) + ' ' + line_productivity['End Time'].astype(str))


In [ ]:
# Adjust End Time for overnight shifts
line_productivity.loc[line_productivity['End Time'] < line_productivity['Start Time'], 'End Time'] += pd.Timedelta(days=1)

In [ ]:
line_productivity['Start Time'] = line_productivity['Start Time'].dt.tz_localize(None)
line_productivity['End Time'] = line_productivity['End Time'].dt.tz_localize(None)
#Handle Timezone Issues


In [ ]:
line_productivity['Duration'] = (line_productivity['End Time'] - line_productivity['Start Time']).dt.total_seconds() / 60  # Convert to minutes
#duration = end time - start time

In [ ]:
line_productivity['Hour'] = line_productivity['Start Time'].dt.hour
line_productivity['Shift'] = line_productivity['Hour'].apply(lambda x: 'Night' if x < 6 else ('Morning' if x < 12 else ('Afternoon' if x < 18 else 'Evening')))
#hour and shift

In [ ]:
line_downtime['Total Downtime'] = line_downtime.iloc[:, 1:].sum(axis=1)
#Calculate Downtime per Batch

In [ ]:
downtime_map = dict(zip(line_downtime['Batch'], line_downtime['Total Downtime']))
line_productivity['Total Downtime'] = line_productivity['Batch'].map(downtime_map)
#Map Total Downtime to the line_productivity DataFrame using the Batch column

In [ ]:
line_productivity['Efficiency'] = ((line_productivity['Duration'] - line_productivity['Total Downtime']) / line_productivity['Duration']) * 100
#Calculate Efficiency

In [ ]:
########### Add Both Factor Numbers and Descriptions ##############

# Create a mapping from factor numbers to descriptions
factor_mapping = dict(zip(downtime_factors['Factor'], downtime_factors['Description']))

# Function to map downtime factors to their descriptions
def map_factors(row):
    # Convert column names to integers for mapping, but only for numeric columns
    factors = [factor_mapping[int(col)] for col in row.index if str(col).isdigit() and row[col] > 0]
    return ', '.join(factors) if factors else 'No downtime'

# Apply the function to each row in the line_downtime DataFrame
line_downtime['Downtime Factors (Descriptions)'] = line_downtime.iloc[:, 1:].apply(map_factors, axis=1)


In [ ]:
#Operator Error
factor_mapping = dict(zip(downtime_factors['Factor'], downtime_factors['Description']))
operator_error_mapping = dict(zip(downtime_factors['Factor'], downtime_factors['Operator Error']))

# Function to map downtime factors to their descriptions and categorize human errors
def map_factors(row):
    factors = []
    human_errors = []
    for col in row.index:
        if str(col).isdigit() and row[col] > 0:
            factors.append(factor_mapping[int(col)])
            human_errors.append(operator_error_mapping[int(col)])
    return ', '.join(factors) if factors else 'No downtime', ', '.join(human_errors) if factors else 'No downtime'

# Apply the function to each row in the line_downtime DataFrame
line_downtime[['Downtime Factors (Descriptions)', 'Human Errors']] = line_downtime.iloc[:, 1:].apply(map_factors, axis=1, result_type='expand')


In [ ]:
#Total Downtime with and without Human Errors
# Create a mapping from factor numbers to descriptions and operator error status
factor_mapping = dict(zip(downtime_factors['Factor'], downtime_factors['Description']))
operator_error_mapping = dict(zip(downtime_factors['Factor'], downtime_factors['Operator Error']))

# Function to calculate total downtime with and without human errors
def calculate_human_error_downtime(row):
    total_with_human = 0
    total_without_human = 0
    for col in row.index:
        if str(col).isdigit() and row[col] > 0:
            if operator_error_mapping[int(col)] == 'Yes':
                total_with_human += row[col]
            else:
                total_without_human += row[col]
    return total_with_human, total_without_human

# Apply the function to each row in the line_downtime DataFrame
line_downtime[['Total Downtime with Human Errors', 'Total Downtime without Human Errors']] = line_downtime.iloc[:, 1:].apply(calculate_human_error_downtime, axis=1, result_type='expand')

In [ ]:
# Create a mapping from product to minimum batch time
min_batch_time_mapping = dict(zip(products['Product'], products['Min batch time']))

# Map minimum batch time to each batch in line_productivity
line_productivity['Min Batch Time'] = line_productivity['Product'].map(min_batch_time_mapping)

# Verify that 'Min Batch Time' column exists
if 'Min Batch Time' not in line_productivity.columns:
    raise KeyError("'Min Batch Time' column not found in line_productivity. Check the mapping.")

# Map total downtime with and without human errors to line_productivity
downtime_map_with_human = dict(zip(line_downtime['Batch'], line_downtime['Total Downtime with Human Errors']))
downtime_map_without_human = dict(zip(line_downtime['Batch'], line_downtime['Total Downtime without Human Errors']))
line_productivity['Total Downtime with Human Errors'] = line_productivity['Batch'].map(downtime_map_with_human)
line_productivity['Total Downtime without Human Errors'] = line_productivity['Batch'].map(downtime_map_without_human)

# Calculate efficiency excluding human errors
line_productivity['Line lagging Efficiency'] = (line_productivity['Min Batch Time'] / (line_productivity['Duration'] - line_productivity['Total Downtime with Human Errors'])) * 100

# Calculate efficiency excluding non-human errors
line_productivity['Opreator lagging Efficiency'] = (line_productivity['Min Batch Time'] / (line_productivity['Duration'] - line_productivity['Total Downtime without Human Errors'])) * 100

# **Export Data**

In [ ]:
downtime_factors.to_excel('downtime_factorsEdited.xlsx', index=False)
line_downtime.to_excel('line_downtimeEdited.xlsx', index=False)
line_productivity.to_excel('line_productivityEdited.xlsx', index=False)
products.to_excel('productsEdited.xlsx', index=False)